In [1]:
import numpy as np
import bilby
from scipy.io import loadmat
from pygwb.pe import *
from pygwb.baseline import Baseline
import bilby.gw.detector as bilbydet

# Parameter Estimation tutorial

In [2]:
#import data
#take e.g. O3
frequencies, Y_HL, sigma_HL = np.loadtxt('/home/katarina.martinovic/o3_data/C_O3_HL.dat', unpack=True, usecols=(0,1,2))
Y_HV, sigma_HV = np.loadtxt('/home/katarina.martinovic/o3_data/C_O3_HV.dat', unpack=True, usecols=(1,2))
Y_LV, sigma_LV = np.loadtxt('/home/katarina.martinovic/o3_data/C_O3_LV.dat', unpack=True, usecols=(1,2))

In [3]:
#go up to 256 Hz
idx=np.argmin(np.abs(frequencies-256))
frequencies = frequencies[:idx]
#cut all of the data for frequencies > 256 Hz
sigma_HL = sigma_HL[:idx]
Y_HL = Y_HL[:idx]
sigma_HV = sigma_HV[:idx]
Y_HV = Y_HV[:idx]
sigma_LV = sigma_LV[:idx]
Y_LV = Y_LV[:idx]

In [4]:
#remove infinities from sigma measurements(O3)
inf_array=np.isinf(sigma_HL)
not_inf_array = ~ inf_array
sigma_HL = sigma_HL[not_inf_array]
Y_HL=Y_HL[not_inf_array]
sigma_HV = sigma_HV[not_inf_array]
Y_HV=Y_HV[not_inf_array]
sigma_LV = sigma_LV[not_inf_array]
Y_LV=Y_LV[not_inf_array]
frequencies=frequencies[not_inf_array]

inf_array=np.isinf(sigma_HV)
not_inf_array = ~ inf_array
sigma_HL = sigma_HL[not_inf_array]
Y_HL=Y_HL[not_inf_array]
sigma_HV = sigma_HV[not_inf_array]
Y_HV=Y_HV[not_inf_array]
sigma_LV = sigma_LV[not_inf_array]
Y_LV=Y_LV[not_inf_array]
frequencies=frequencies[not_inf_array]

inf_array=np.isinf(sigma_LV)
not_inf_array = ~ inf_array
sigma_HL = sigma_HL[not_inf_array]
Y_HL=Y_HL[not_inf_array]
sigma_HV = sigma_HV[not_inf_array]
Y_HV=Y_HV[not_inf_array]
sigma_LV = sigma_LV[not_inf_array]
Y_LV=Y_LV[not_inf_array]
frequencies=frequencies[not_inf_array]

In [5]:
point_estimates = [Y_HL, Y_HV, Y_LV]
sigmas = [sigma_HL, sigma_HV, sigma_LV]
H1 = bilbydet.get_empty_interferometer('H1')
L1 = bilbydet.get_empty_interferometer('L1')
V1 = bilbydet.get_empty_interferometer('V1')

HL = Baseline('H1L1', H1, L1)
HV = Baseline('H1V1', H1, V1)
LV = Baseline('L1V1', L1, V1)

HL.frequencies = frequencies
HV.frequencies = frequencies
LV.frequencies = frequencies


HL.point_estimate = Y_HL
HL.sigma = sigma_HL
HV.point_estimate = Y_HV
HV.sigma = sigma_HV
LV.point_estimate = Y_LV
LV.sigma = sigma_LV

#take the more conservative O3b calibration
HL.calibration_epsilon = 0.148
HV.calibration_epsilon = 0.123
LV.calibration_epsilon = 0.108

/home/arianna.renzini/PROJECTS/pygwb/pygwb/baseline.py:249: UserWarning: Neither baseline nor interferometer duration is set.
  warnings.warn("Neither baseline nor interferometer duration is set.")
/home/arianna.renzini/PROJECTS/pygwb/pygwb/baseline.py:446: UserWarning: Neither baseline nor interferometer sampling_frequency is set.
  warnings.warn(


In [6]:
###############################################
###############Testing pl##################
###############################################

#choose pair likelihoods for the models you want to constrain with the data
kwargs_pl = {"baselines":[HL,HV,LV], "model_name":'PL', "fref":25}
model_pl = PowerLawModel(**kwargs_pl)
priors_pl = {'omega_ref': bilby.core.prior.LogUniform(1e-13, 1e-5, '$\\Omega_{\\rm ref}$'),
                        'alpha': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha$')}
hlv_pl=bilby.run_sampler(likelihood=model_pl,priors=priors_pl,sampler='dynesty', npoints=1000, walks=10, npool=10, outdir='./pe/',label= 'hlv_pl', resume=False)
hlv_pl.plot_corner()

AttributeError: 'Baseline' object has no attribute '_orf_polarization'

In [ ]:
# ###############################################
# ###############Testing bpl##################
# ###############################################

# kwargs_bpl = {"baselines": [HL, HV, LV], "model_name":'BPL'}
# model_bpl = BrokenPowerLawModel(**kwargs_bpl)
# priors_bpl = {'omega_ref': bilby.core.prior.LogUniform(1e-13, 1e-5, '$\\Omega_{\\rm ref}$'),
#             'fbreak': bilby.core.prior.Uniform(1, 100,'$f_{\\rm break}$'),
#             'alpha_1': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha_1$'),
#             'alpha_2': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha_2$')}
# hlv_bpl=bilby.run_sampler(likelihood=model_bpl,priors=priors_bpl,sampler='dynesty', npoints=1000, walks=10,npool=10,outdir='./',label= 'hlv_bpl', resume=False)
# hlv_bpl.plot_corner()

In [ ]:
###############################################
######### Testing triple_BPL##################
###############################################

# kwargs_triple_bpl = {"baselines":[HL,HV,LV],"model_name": 'TBPL'}
# model_triple_bpl = TripleBrokenPowerLawModel(**kwargs_triple_bpl)
# priors_triple_bpl = {'omega_ref': bilby.core.prior.LogUniform(1e-13, 1e-5, '$\\Omega_{\\rm ref}$'),
#                        'alpha_1': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha_1$'),
#                       'alpha_2': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha_2$'),
#                        'alpha_3': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha_3$'),
#                      'fbreak1': bilby.core.prior.Uniform(1, 100,'$f_{\\rm break}^1$'), 
#                      'fbreak2': bilby.core.prior.Uniform(1, 100,'$f_{\\rm break}^2$')}
# hlv_triple_bpl = bilby.run_sampler(likelihood=model_triple_bpl,priors=priors_triple_bpl,sampler='dynesty', npoints=1000, walks=10,npool=10,outdir='./',label= 'hlv_tbpl', resume=False)
# hlv_triple_bpl.plot_corner()

In [ ]:
# ###############################################
# ######### Testing Smooth Broken PL#############
# ###############################################

# kwargs_sbpl = {"baselines":[HL,HV,LV],"model_name": 'SBPL'}
# model_sbpl = SmoothBrokenPowerLawModel(**kwargs_sbpl)
# priors_sbpl = {'omega_ref': bilby.core.prior.LogUniform(1e-13, 1e-5, '$\\Omega_{\\rm ref}$'),
#                   'fbreak': bilby.core.prior.Uniform(1, 256, '$f_{\\rm break}$'),
#                         'alpha_1': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha_1$'),
#                        'alpha_2': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha_2$'),
#                         'delta': bilby.core.prior.Uniform(0, 8, '$\\Delta$')}
# hlv_sbpl = bilby.run_sampler(likelihood=model_sbpl,priors=priors_sbpl,sampler='dynesty', npoints=1000, walks=10,npool=10,outdir='./',label= 'hlv_sbpl', resume=False)
# hlv_sbpl.plot_corner()

In [ ]:
# ###############################################
# ###############Testing tvs pl##################
# ###############################################

# kwargs_pl_sv={"baselines":[HL, HV, LV], "model_name":'PL_SV', "fref":25, "polarizations":['scalar', 'vector']}
# model_pl_sv = TVSPowerLawModel(**kwargs_pl_sv)
# priors_pl_sv = {'omega_ref_scalar': bilby.core.prior.LogUniform(1e-13, 1e-5, '$\\Omega_{\\rm ref,s}$'),
#                       'alpha_scalar': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha_s$'),
#           'omega_ref_vector': bilby.core.prior.LogUniform(1e-13, 1e-5, '$\\Omega_{\\rm ref,v}$'),
#                       'alpha_vector': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha_v$')}
# hlv_pl_sv=bilby.run_sampler(likelihood=model_pl_sv,priors=priors_pl_sv,sampler='dynesty', npoints=1000, walks=10,npool=10,outdir='./',label= 'hlv_pl_sv', resume=False)
# hlv_pl_sv.plot_corner()

In [ ]:
# ###############################################
# ######### Testing Parity Violation PL 1 #######
# ###############################################

# kwargs_pl_pv = {"baselines":[HL, HV, LV],"model_name": 'PL_PV', 'fref': 25}
# model_pl_pv = PVPowerLawModel(**kwargs_pl_pv)
# priors_pl_pv = {'omega_ref': bilby.core.prior.LogUniform(1e-13, 1e-5, '$\\Omega_{\\rm ref}$'),
#                        'alpha': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha$'),
#                        'Pi': bilby.core.prior.Uniform(-1,1,'$\\Pi$')}
# hlv_pl_pv = bilby.run_sampler(likelihood=model_pl_pv,priors=priors_pl_pv,sampler='dynesty', npoints=1000, walks=10,npool=10,outdir='./',label= 'hlv_pl_pv', resume=False)
# hlv_pl_pv.plot_corner()

In [ ]:
##############################################
######## Testing Parity Violation PL 2 #######
##############################################

# kwargs_pv_pl_2 = {"baselines":[HL, HV, LV],"model_name": 'PL_PV_2', 'fref': 25}
# model_pv_pl_2 = PVPowerLawModel2(**kwargs_pv_pl_2)
# priors_pv_pl_2 = {'omega_ref': bilby.core.prior.LogUniform(1e-13, 1e-5, '$\\Omega_{\\rm ref}$'),
#                      'alpha': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha$'),
#                        'beta': bilby.core.prior.Uniform(-2,0,'$\\beta$')}
# hlv_pv_pl_2 = bilby.run_sampler(likelihood=model_pv_pl_2,priors=priors_pv_pl_2,sampler='dynesty', npoints=1000, walks=10,npool=10,outdir='./',label= 'hlv_pv_pl_2', resume=False)
# hlv_pv_pl_2.plot_corner()

In [ ]:
# ##############################################
# ######## Testing Schumann #######
# ##############################################

# frequencies, Y_HL, sigma_HL = np.loadtxt('C_O3_HL.dat', unpack=True, usecols=(0,1,2))
# Y_HV, sigma_HV = np.loadtxt('C_O3_HV.dat', unpack=True, usecols=(1,2))
# Y_LV, sigma_LV = np.loadtxt('C_O3_LV.dat', unpack=True, usecols=(1,2))
# #load magnetic data
# mag_data = loadmat('/home/patrick.meyers/git_repos/o3-isotropic-final/magnetic/magnetic_data_product_for_pe.mat')  
# M_HL=(mag_data['hl']['ptEst_mag'][0][0].real)[0]
# M_HV=(mag_data['hv']['ptEst_mag'][0][0].real)[0]
# M_LV=(mag_data['lv']['ptEst_mag'][0][0].real)[0]

# #go up to 99 Hz
# idx=np.argmin(np.abs(frequencies-99))
# frequencies = frequencies[:idx]
# #cut all of the data for frequencies > 99 Hz
# sigma_HL = sigma_HL[:idx]
# Y_HL = Y_HL[:idx]
# M_HL = M_HL[:idx]
# sigma_HV = sigma_HV[:idx]
# Y_HV = Y_HV[:idx]
# M_HV = M_HV[:idx]
# sigma_LV = sigma_LV[:idx]
# Y_LV = Y_LV[:idx]
# M_LV = M_LV[:idx]

# #remove infinities from sigma measurements(O3a)
# inf_array=np.isinf(sigma_HL)
# not_inf_array = ~ inf_array
# sigma_HL = sigma_HL[not_inf_array]
# Y_HL=Y_HL[not_inf_array]
# M_HL=M_HL[not_inf_array]
# sigma_HV = sigma_HV[not_inf_array]
# Y_HV=Y_HV[not_inf_array]
# M_HV=M_HV[not_inf_array]
# sigma_LV = sigma_LV[not_inf_array]
# Y_LV=Y_LV[not_inf_array]
# M_LV=M_LV[not_inf_array]
# frequencies=frequencies[not_inf_array]

# inf_array=np.isinf(sigma_HV)
# not_inf_array = ~ inf_array
# sigma_HL = sigma_HL[not_inf_array]
# Y_HL=Y_HL[not_inf_array]
# M_HL=M_HL[not_inf_array]
# sigma_HV = sigma_HV[not_inf_array]
# Y_HV=Y_HV[not_inf_array]
# M_HV=M_HV[not_inf_array]
# sigma_LV = sigma_LV[not_inf_array]
# Y_LV=Y_LV[not_inf_array]
# M_LV=M_LV[not_inf_array]
# frequencies=frequencies[not_inf_array]

# inf_array=np.isinf(sigma_LV)
# not_inf_array = ~ inf_array
# sigma_HL = sigma_HL[not_inf_array]
# Y_HL=Y_HL[not_inf_array]
# M_HL=M_HL[not_inf_array]
# sigma_HV = sigma_HV[not_inf_array]
# Y_HV=Y_HV[not_inf_array]
# M_HV=M_HV[not_inf_array]
# sigma_LV = sigma_LV[not_inf_array]
# Y_LV=Y_LV[not_inf_array]
# M_LV=M_LV[not_inf_array]
# frequencies=frequencies[not_inf_array]

# #remove nans from mag data
# nan_array = np.isnan(M_HL)
# not_nan_array = ~ nan_array
# sigma_HL = sigma_HL[not_nan_array]
# Y_HL=Y_HL[not_nan_array]
# M_HL=M_HL[not_nan_array]
# sigma_HV = sigma_HV[not_nan_array]
# Y_HV=Y_HV[not_nan_array]
# M_HV=M_HV[not_nan_array]
# sigma_LV = sigma_LV[not_nan_array]
# Y_LV=Y_LV[not_nan_array]
# M_LV=M_LV[not_nan_array]
# frequencies=frequencies[not_nan_array]

# nan_array = np.isnan(M_HV)
# not_nan_array = ~ nan_array
# sigma_HL = sigma_HL[not_nan_array]
# Y_HL=Y_HL[not_nan_array]
# M_HL=M_HL[not_nan_array]
# sigma_HV = sigma_HV[not_nan_array]
# Y_HV=Y_HV[not_nan_array]
# M_HV=M_HV[not_nan_array]
# sigma_LV = sigma_LV[not_nan_array]
# Y_LV=Y_LV[not_nan_array]
# M_LV=M_LV[not_nan_array]
# frequencies=frequencies[not_nan_array]

# nan_array = np.isnan(M_LV)
# not_nan_array = ~ nan_array
# sigma_HL = sigma_HL[not_nan_array]
# Y_HL=Y_HL[not_nan_array]
# M_HL=M_HL[not_nan_array]
# sigma_HV = sigma_HV[not_nan_array]
# Y_HV=Y_HV[not_nan_array]
# M_HV=M_HV[not_nan_array]
# sigma_LV = sigma_LV[not_nan_array]
# Y_LV=Y_LV[not_nan_array]
# M_LV=M_LV[not_nan_array]
# frequencies=frequencies[not_nan_array]

# point_estimates = [Y_HL, Y_HV, Y_LV]
# sigmas = [sigma_HL, sigma_HV, sigma_LV]
# H1 = bilbydet.get_empty_interferometer('H1')
# L1 = bilbydet.get_empty_interferometer('L1')
# V1 = bilbydet.get_empty_interferometer('V1')

# HL = Baseline('HL', H1, L1)
# HV = Baseline('HV', H1, V1)
# LV = Baseline('LV', L1, V1)

# HL.frequencies = frequencies
# HV.frequencies = frequencies
# LV.frequencies = frequencies

# HL.M_f = M_HL
# HV.M_f = M_HV
# LV.M_f = M_LV

# HL.point_estimate = Y_HL
# HL.sigma = sigma_HL
# HV.point_estimate = Y_HV
# HV.sigma = sigma_HV
# LV.point_estimate = Y_LV
# LV.sigma = sigma_LV

# kwargs_schu={"baselines":[HL, HV, LV], "model_name":'Schu', "polarizations":['tensor']}
# model_schu = SchumannModel(**kwargs_schu)
# print(model_schu.ifos)
# priors_schu = {'kappa_H': bilby.core.prior.LogUniform(0.01, 10, '$\\kappa_H$'),
#             'kappa_L': bilby.core.prior.LogUniform(0.01, 10, '$\\kappa_L$'),
#             'kappa_V': bilby.core.prior.LogUniform(0.01, 10, '$\\kappa_V$'),
#             'beta_H': bilby.core.prior.Uniform(0.0, 12.0, '$\\beta_H$'),
#             'beta_L': bilby.core.prior.Uniform(1.0, 10.0, '$\\beta_L$'),
#            'beta_V': bilby.core.prior.Uniform(0.0, 7.0, '$\\beta_V$')}
# hlv_schu=bilby.run_sampler(likelihood=model_schu,priors=priors_schu,sampler='dynesty', npoints=1000, walks=10,npool=10,outdir='./',label= 'hlv_schu', resume=False)
# hlv_schu.plot_corner()

In [ ]:
# # ##############################################
# # ######## Testing Schumann + pl #######
# # ##############################################
# #########combining models
# #https://stackoverflow.com/questions/9667818/python-how-to-merge-two-class

# #import data
# #take e.g. O3
# frequencies, Y_HL, sigma_HL = np.loadtxt('C_O3_HL.dat', unpack=True, usecols=(0,1,2))
# Y_HV, sigma_HV = np.loadtxt('C_O3_HV.dat', unpack=True, usecols=(1,2))
# Y_LV, sigma_LV = np.loadtxt('C_O3_LV.dat', unpack=True, usecols=(1,2))
# #load magnetic data
# mag_data = loadmat('/home/patrick.meyers/git_repos/o3-isotropic-final/magnetic/magnetic_data_product_for_pe.mat')  
# M_HL=(mag_data['hl']['ptEst_mag'][0][0].real)[0]
# M_HV=(mag_data['hv']['ptEst_mag'][0][0].real)[0]
# M_LV=(mag_data['lv']['ptEst_mag'][0][0].real)[0]

# #go up to 99 Hz
# idx=np.argmin(np.abs(frequencies-99))
# frequencies = frequencies[:idx]
# #cut all of the data for frequencies > 99 Hz
# sigma_HL = sigma_HL[:idx]
# Y_HL = Y_HL[:idx]
# M_HL = M_HL[:idx]
# sigma_HV = sigma_HV[:idx]
# Y_HV = Y_HV[:idx]
# M_HV = M_HV[:idx]
# sigma_LV = sigma_LV[:idx]
# Y_LV = Y_LV[:idx]
# M_LV = M_LV[:idx]

# #remove infinities from sigma measurements(O3a)
# inf_array=np.isinf(sigma_HL)
# not_inf_array = ~ inf_array
# sigma_HL = sigma_HL[not_inf_array]
# Y_HL=Y_HL[not_inf_array]
# M_HL=M_HL[not_inf_array]
# sigma_HV = sigma_HV[not_inf_array]
# Y_HV=Y_HV[not_inf_array]
# M_HV=M_HV[not_inf_array]
# sigma_LV = sigma_LV[not_inf_array]
# Y_LV=Y_LV[not_inf_array]
# M_LV=M_LV[not_inf_array]
# frequencies=frequencies[not_inf_array]

# inf_array=np.isinf(sigma_HV)
# not_inf_array = ~ inf_array
# sigma_HL = sigma_HL[not_inf_array]
# Y_HL=Y_HL[not_inf_array]
# M_HL=M_HL[not_inf_array]
# sigma_HV = sigma_HV[not_inf_array]
# Y_HV=Y_HV[not_inf_array]
# M_HV=M_HV[not_inf_array]
# sigma_LV = sigma_LV[not_inf_array]
# Y_LV=Y_LV[not_inf_array]
# M_LV=M_LV[not_inf_array]
# frequencies=frequencies[not_inf_array]

# inf_array=np.isinf(sigma_LV)
# not_inf_array = ~ inf_array
# sigma_HL = sigma_HL[not_inf_array]
# Y_HL=Y_HL[not_inf_array]
# M_HL=M_HL[not_inf_array]
# sigma_HV = sigma_HV[not_inf_array]
# Y_HV=Y_HV[not_inf_array]
# M_HV=M_HV[not_inf_array]
# sigma_LV = sigma_LV[not_inf_array]
# Y_LV=Y_LV[not_inf_array]
# M_LV=M_LV[not_inf_array]
# frequencies=frequencies[not_inf_array]

# #remove nans from mag data
# nan_array = np.isnan(M_HL)
# not_nan_array = ~ nan_array
# sigma_HL = sigma_HL[not_nan_array]
# Y_HL=Y_HL[not_nan_array]
# M_HL=M_HL[not_nan_array]
# sigma_HV = sigma_HV[not_nan_array]
# Y_HV=Y_HV[not_nan_array]
# M_HV=M_HV[not_nan_array]
# sigma_LV = sigma_LV[not_nan_array]
# Y_LV=Y_LV[not_nan_array]
# M_LV=M_LV[not_nan_array]
# frequencies=frequencies[not_nan_array]

# nan_array = np.isnan(M_HV)
# not_nan_array = ~ nan_array
# sigma_HL = sigma_HL[not_nan_array]
# Y_HL=Y_HL[not_nan_array]
# M_HL=M_HL[not_nan_array]
# sigma_HV = sigma_HV[not_nan_array]
# Y_HV=Y_HV[not_nan_array]
# M_HV=M_HV[not_nan_array]
# sigma_LV = sigma_LV[not_nan_array]
# Y_LV=Y_LV[not_nan_array]
# M_LV=M_LV[not_nan_array]
# frequencies=frequencies[not_nan_array]

# nan_array = np.isnan(M_LV)
# not_nan_array = ~ nan_array
# sigma_HL = sigma_HL[not_nan_array]
# Y_HL=Y_HL[not_nan_array]
# M_HL=M_HL[not_nan_array]
# sigma_HV = sigma_HV[not_nan_array]
# Y_HV=Y_HV[not_nan_array]
# M_HV=M_HV[not_nan_array]
# sigma_LV = sigma_LV[not_nan_array]
# Y_LV=Y_LV[not_nan_array]
# M_LV=M_LV[not_nan_array]
# frequencies=frequencies[not_nan_array]

# point_estimates = [Y_HL, Y_HV, Y_LV]
# sigmas = [sigma_HL, sigma_HV, sigma_LV]
# H1 = bilbydet.get_empty_interferometer('H1')
# L1 = bilbydet.get_empty_interferometer('L1')
# V1 = bilbydet.get_empty_interferometer('V1')

# HL = Baseline('HL', H1, L1)
# HV = Baseline('HV', H1, V1)
# LV = Baseline('LV', L1, V1)

# HL.frequencies = frequencies
# HV.frequencies = frequencies
# LV.frequencies = frequencies

# HL.M_f = M_HL
# HV.M_f = M_HV
# LV.M_f = M_LV

# HL.point_estimate = Y_HL
# HL.sigma = sigma_HL
# HV.point_estimate = Y_HV
# HV.sigma = sigma_HV
# LV.point_estimate = Y_LV
# LV.sigma = sigma_LV

# gw_schu = type('gw_schu', (PowerLawModel,SchumannModel), dict(c='c'))
# #instc = pl_schu('kwargs')
# kwargs={"baselines":[HL, HV, LV], "model_name":'PL+Schu', "polarizations":['tensor'],"fref":25}
# model = gw_schu(**kwargs)

# priors= {'kappa_H': bilby.core.prior.LogUniform(0.01, 10, '$\\kappa_H$'),
#             'kappa_L': bilby.core.prior.LogUniform(0.01, 10, '$\\kappa_L$'),
#             'kappa_V': bilby.core.prior.LogUniform(0.01, 10, '$\\kappa_V$'),
#             'beta_H': bilby.core.prior.Uniform(0.0, 12.0, '$\\beta_H$'),
#             'beta_L': bilby.core.prior.Uniform(1.0, 10.0, '$\\beta_L$'),
#            'beta_V': bilby.core.prior.Uniform(0.0, 7.0, '$\\beta_V$'),
#             'omega_ref': bilby.core.prior.LogUniform(1e-13, 1e-5, '$\\Omega_{\\rm ref}$'),
#              'alpha': bilby.core.prior.Gaussian(0, 3.5, '$\\alpha$')}
# hlv_gw_schu=bilby.run_sampler(likelihood=model,priors=priors,sampler='dynesty', npoints=1000, walks=10,npool=10,outdir='./',label= 'hlv_gw_schu', resume=False)
# hlv_gw_schu.plot_corner()

Task was destroyed but it is pending!
task: <Task pending name='Task-2' coro=<Kernel.poll_control_queue() running at /home/arianna.renzini/.conda/envs/pygwb/lib/python3.8/site-packages/ipykernel/kernelbase.py:243> wait_for=<Future finished result=[<zmq.sugar.fr...x7f5596c7cb40>, <zmq.sugar.fr...x7f5596be2720>, <zmq.sugar.fr...x7f58763e1300>, <zmq.sugar.fr...x7f58680db250>, <zmq.sugar.fr...x7f58680db3b0>, <zmq.sugar.fr...x7f58680db300>, ...]> cb=[_chain_future.<locals>._call_set_state() at /home/arianna.renzini/.conda/envs/pygwb/lib/python3.8/asyncio/futures.py:367]>
